<h1 align="center"><font color="yellow">Quatro maneiras de Question-Answering in LangChain</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Este script está baseado nos trabalhos da [Sophia Yang]()

# Contextualizando

Converse com seus longos documentos `PDF`: load_qa_chain, RetrievalQA, VectorstoreIndexCreator, ConversationalRetrievalChain.


Você está interessado em conversar com seus próprios documentos, seja um `arquivo de texto`, um `PDF` ou um `site`? O `LangChain` torna mais fácil para você responder a perguntas com seus documentos. Mas você sabia que existem pelo menos `4` maneiras de responder a perguntas no LangChain? Nesta postagem do blog, exploraremos quatro maneiras diferentes de responder a perguntas e as várias opções que você pode considerar para seus casos de uso.

Só lembrando que é `LangChain`, na opinião de `Sophia Yang`, é a maneira mais fácil de interagir com modelos de linguagem e criar aplicativos. É uma ferramenta de código aberto que envolve muitos `LLMs` e ferramentas. 

Ok, agora vamos começar a responder perguntas em documentos externos.


# Configurando a API da OpenAI e importando as Bibliotecas necessárias

Observe que a [API OpenAI](https://platform.openai.com/account) não é gratuita. Você precisará configurar as informações de cobrança para poder usar a `API OpenAI`. Como alternativa, você pode usar modelos do `HuggingFace Hub` ou de outros lugares.


In [ ]:
#%pip install langchain==0.0.137 openai chromadb tiktoken pypdf

In [1]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 

Carregando a minha chave Key:  True


In [2]:
from langchain.chains import RetrievalQA # Tem que atualizar --> pip install langchai==0.0.137
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


# Carregando Documentos

O `LangChain` suporta muitos [carregadores de documentos](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html), como `Notion`, `YouTube` e `Figma`. Neste exemplo, gostaria de conversar com meu arquivo `PDF`. Assim, usei o `PyPDFLoader` para carregar meu arquivo.

Eu recomendo usar um PDF pequeno. Eu usei, só para demonstração mesmo, um boleto de pagamento da minha conta de energia que contém apenas uma página (você pode usar qualquer outro documento). Meu boleto contém certas entidades e em base a elas farei certas perguntas! 


In [3]:
# Verificamos nosso Modelo Instanciado:

llm = OpenAI()
print(llm("Conte-me uma piada."))




O que aconteceu quando o tijolo caiu do céu? Ele deu um tijolo no chão.


In [4]:
# Carregando meu documento (Boleto de energia):
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./Karina_boleto_FACULDADE.pdf")
documents = loader.load()

In [7]:
documents

[Document(page_content=' \nRECIBO DO PAGADOR\nBeneficiário\nUNIFRAN - GRADUAÇÃO EAD - CNPJ: 046.722.831/0001-78 \nAv. Dr. Armando Salles Oliveira - Franca/SPAgência/Cód. Beneficiário\n3393 - 6 / 0001544 - PData do Documento\n02/03/2023Vencimento\n27/03/2023\nPagador\nKARINA GONÇALVES SOARESNúmero Documento\n23103/06Nosso Número\n19/00236419200-6Valor do Documento\nR$ 551,26\nInstruções: (Todas as informações deste bloqueto são de exclusiva responsabilidade do beneficiário)\n* SR. CAIXA NAO RECEBER APOS VENCIMENTO; \n* DESCONTO DE R$ 320,43 ATE 07/03/23; R$ 289,64 ATE 27/03/23; \n* (3A CST EM CIÊNCIA DE DADOS) - RGM:22-30419841 \nAutenticação Mecânica\n237-2 23793.39316 90023.641922 00000.154401 2 93020000055126\nLocal de Pagamento\nPagável preferencialmente nas agências BRADESCOVencimento\n27/03/2023\nBeneficiário\nUNIFRAN - GRADUAÇÃO EAD - CNPJ: 046.722.831/0001-78 - Av. Dr. Armando Salles Oliveira - Franca/SPAgência/Código Beneficiário\n3393 - 6 / 0001544 - P\nData do Documento\n02/0

# Método 1: <font color="red">load_qa_chain</font>

`load_qa_chain` fornece a interface mais genérica para responder a perguntas. Ele carrega uma cadeia que você pode fazer `QA` para seus documentos de entrada e usa TODO o texto nos documentos.

`chain_type="stuff"` não funcionará porque o número de tokens excede o limite. Podemos tentar outros tipos de cadeia como `"map_reduce"`.

In [8]:
from langchain.chains.question_answering import load_qa_chain


chain = load_qa_chain(llm=OpenAI(),
                      chain_type="map_reduce"
                     )


query = "Qual é o nome do pagador?"

chain.run(input_documents=documents,
          question=query
         )


' O nome do pagador é KARINA GONÇALVES SOARES.'

<font color="orange">Ele também permite que você faça QA em um conjunto de documentos:</font>

In [ ]:
### Para vários Documentos:
# loaders = [....]
# documents = []
# for loader in loaders:
#     documents.extend(loader.load())

🤔 Mas e se meu documento for muito longo e ultrapassar o limite de token?

Existem duas maneiras de corrigi-lo:

## <font color="pink">Solução 1: Tipo de Cadeia (chain)</font>

O padrão `chain_type="stuff"` usa `TODO` o texto dos documentos no prompt. Se você tiver um arquivo muito grande (`tal como fez Sophia Yang, com um PDF de `50` páginas`) não funcionará porque excede o `limite de token` e causa erros de limitação de taxa. É por isso que para o exemplo da Sophia Yang, ela teve que usar outro tipo de cadeia, por exemplo `"map_reduce"`. 

A seguir vamos estudar outros tipos de cadeias:

🤩 `map_reduce:` Separa os textos em lotes (<font color="red">por exemplo</font>, você pode definir o tamanho do lote em `llm=OpenAI(batch_size=5)`), alimenta cada lote com a pergunta para o `LLM` separadamente e apresenta a resposta final com base nas respostas de cada lote.

🤩 `refine:` Separa os textos em lotes, alimenta o primeiro lote para o LLM e alimenta a resposta e o segundo lote para o LLM. Ele refina a resposta passando por todos os lotes.

🤩 `map-rerank:` ele separa os textos em lotes, alimenta cada lote para o LLM, retorna uma pontuação de como responde completamente à pergunta e apresenta a resposta final com base nas respostas com pontuação mais alta de cada lote.

## <font color="pink">Solução 2: Recuperação QA (`RetrievalQA`)</font>

Um problema com o uso de `TODO` o texto é que pode ser `muito caro` porque você está alimentando todos os textos para a `API OpenAI` e a `API` é cobrada pelo número de tokens. Uma solução melhor é recuperar os blocos (`chunks`) de texto relevantes primeiro e usar apenas os blocos de texto relevantes no modelo de linguagem. A seguir, examinarei os detalhes do `RetrievalQA`.

# Método 2: <font color="red">RetrievalQA</font>